In [ ]:
#pip install pandas
#pip install matplotlib
#!pip install seaborn
#!pip install nbconvert[webpdf]

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

from scipy.stats import norm 
from scipy import stats 